<a href="https://colab.research.google.com/github/giuli-raffin/Sistema-de-Recomendacion/blob/main/recomendacion-profesores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Final Sistemas de Recomendación

In [ ]:
pip install pandas


In [ ]:
import pandas as pd

#leer el archivo
#url= 'https://docs.google.com/spreadsheets/d/1LdlVwpZZrYQS19eUzCFhYXcGt-XG1hP67w1A0CmoPHI/edit?gid=1183086110#gid=1183086110'
df_formulario= pd.read_csv('RespuestasFormulario1.csv')

In [ ]:
id_profesor = 6
input_claves = "razones trigonometricas"
input_anio = "7"

##Filtrado Basado en Contenido y Colaborativo basado en memoria Item-Item


### Declaración de matriz de ratings y procesamiento de matriz de problemas

In [ ]:
#creacion de la matriz de problemas
df_problemas= df_formulario.iloc[:,8:14]

#cargar los problemas adicionales que tengan
for i in range(df_formulario.shape[0]):
  j=14
  while(j<df_formulario.shape[1] and df_formulario.iloc[i,j]=='Si'):
    nueva_fila = df_formulario.iloc[i, j+1:j+7]
    nueva_fila_df = nueva_fila.to_frame().T
    nueva_fila_df.columns = df_problemas.columns
    df_problemas = pd.concat([df_problemas, nueva_fila_df], ignore_index=True)
    j=j+7


#cambiamos el nombre de las columnas para mayor comprension
df_problemas.columns=['anio', 'conocimientos previos', 'claves', 'enunciado', 'imagen', 'solucion']

#intercambiar los NaN por un espacio en blanco
for problema in df_problemas:
    df_problemas[problema] = df_problemas[problema].fillna('')

nf = df_problemas.iloc[0]
nf_df = nf.to_frame().T
nf_df.columns = df_problemas.columns
df_problemas = pd.concat([df_problemas,nf_df], ignore_index=True)

df_problemas

,anio,conocimientos previos,claves,enunciado,imagen,solucion
0,6 (estudiantes de 17-18 años),Razones trigonométricas,Ángulos orientados. Razones trigonométricas,Dar las 3 razones trigonométricas principales ...,,
1,6 (estudiantes de 17-18 años),"Circunferencia, plano cartesiano","Los estudiantes, para poder resolver el proble...","Para calcular la altura de un árbol, una perso...",https://drive.google.com/open?id=1mlxNyFQbzSFx...,
2,6 (estudiantes de 17-18 años),Razones trigonométricas,Ángulos centrados: sentido y orientación. Sist...,Representar en un circunferencia de radio 1 la...,,
3,6 (estudiantes de 17-18 años),Concepto de función. Ubicación de puntos en el...,Razones trigonometricas,Graficar y realizar el análisis completo de la...,,
4,6 (estudiantes de 17-18 años),Razones trigonométricas,Ángulos orientados. Razones trigonométricas,Dar las 3 razones trigonométricas principales ...,,


Generador de ratings aleatorios

In [ ]:
import numpy as np

#creacion de la matriz de ratings
num_columnas= df_problemas.shape[0]
num_filas= 10

#creamos un arreglo de probabilidades para cargar la matriz en base a eso
valores = np.arange(0, 11)
probabilidades = [0.6, 0.04, 0.04, 0.04, 0.04, 0.05, 0.04, 0.04, 0.04, 0.04, 0.03]

df_ratings= pd.DataFrame(np.random.choice(valores, size=(num_filas, num_columnas), p=probabilidades),
                  columns=[f'Problema {i+1}' for i in range(num_columnas)])

df_ratings

,Problema 1,Problema 2,Problema 3,Problema 4,Problema 5
0,2,0,0,2,4
1,0,0,0,8,0
2,1,0,0,0,0
3,8,0,0,0,0
4,6,7,0,0,4
5,0,0,0,0,0
6,0,8,0,0,5
7,0,0,0,8,0
8,0,7,9,3,0
9,0,3,0,0,0


### Procesamiento de las claves de los problemas y similaridad del coseno

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cvc = CountVectorizer(strip_accents='ascii' , lowercase= True,token_pattern=r'\b\d+\b|\b\w+\b')
clave_matrix = cvc.fit_transform(df_problemas['claves'])

clave_matrix.toarray()

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 1, 1, 1, 0],
       [0, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0]])

In [ ]:
from scipy.sparse import csr_matrix, vstack
from sklearn.metrics.pairwise import cosine_similarity

#vectorizamos el input de las claves deseadas con relacion a las claves cargadas en el dataframe df_problemas
input_vector= cvc.transform([input_claves])
input_vector.toarray()

#relacionar resultados
#se crea una nueva matriz con las claves vectorizadas y a lo ultimo el input vectorizado
cm = vstack([clave_matrix, input_vector])
cs_clavesnuevas= cosine_similarity(cm)

def mayorSimilitud(matrix): #mayor similitud para anio y ratings
  rango = 0.65
  index = []
  filaN=matrix.shape[1]-1
  for i in range(matrix.shape[0]-1):
    if(rango <= matrix[filaN][i]):
      index.append(i)
  if len(index) == 0:
    index.append(-1)
  return index

def mayorSimilitudPeso(matrix): #mayor similitud para claves
  index = []
  peso = []
  filaN=matrix.shape[1]-1
  lista = matrix[filaN]
  lista = sorted(set(lista), reverse=True)
  if lista[0] > 0.99:
    rango = lista[1]
  else:
    rango = lista[0]
  for i in range(matrix.shape[0]-1):
    if(rango <= matrix[filaN][i]):
      index.append(i)
      if(matrix[filaN][i] > 0.99):
        peso.append(4)
      else:
        peso.append(3)
  if len(index) == 0:
    index.append(-1)
  return index,peso

### Procesamiento del año de dictado del problema

Detección de el/los año/s similar/es

In [ ]:
cvc2 = CountVectorizer(strip_accents='ascii' , lowercase= True, token_pattern=r'\b\d+\b|\b\w+\b')
anio_matrix = cvc2.fit_transform(df_problemas['anio'])

#vectorizamos el input del anio deseado con relacion a los anios cargados en el dataframe df_problemas
input_aniov= cvc2.transform([input_anio])
input_aniov.toarray()

am = vstack([anio_matrix, input_aniov])
cs_anionuevo= cosine_similarity(am)

def cosine_similarity_anio(index):
  anio_matrix = df_problemas.loc[index,'anio']
  cvc2 = CountVectorizer(strip_accents='ascii' , lowercase= True, token_pattern=r'\b\d+\b|\b\w+\b')
  anio_matrix_cv = cvc2.fit_transform(anio_matrix)

  input_aniov = cvc2.transform([input_anio])

  am = vstack([anio_matrix_cv, input_aniov])
  cs_anionuevo = cosine_similarity(am)
  return mayorSimilitud(cs_anionuevo)

### Procesamiento de los ratings realizados por los usuarios sobre los problemas

In [ ]:
df_ratingsT= df_ratings.T
cs_ratings = cosine_similarity(df_ratingsT)


def transform_ps(arr_a, arr_b): #arr_a es anio y arr_b es problema similar
  result = []
  for i in range(len(arr_a)):
    result.insert(i, arr_b[arr_a[i]])
  return result

def cosine_similarity_ratings(index): #en index estan los indices que referencian al arreglo de posibles problemas. devuelve los problemas a recomendar segun los ratings
  index_r = mayorSimilitud(cs_ratings) #todos los posibles segun los ratings

  #que este en problema similar
  for i in range(len(index_r)):
    if (index_r[i] not in index): #si no es posible segun claves y anio lo elimina
      index_r[i] = -1
  index_r = [x for x in index_r if x != -1] #elimina los problemas que no estan en los problemas posibles a recomendar
  return index_r


cs_ratings

array([[1.        , 0.31344156, 0.        , 0.03287424, 0.41363559],
       [0.31344156, 1.        , 0.53530338, 0.13524203, 0.68876874],
       [0.        , 0.53530338, 1.        , 0.25264558, 0.        ],
       [0.03287424, 0.13524203, 0.25264558, 1.        , 0.0892366 ],
       [0.41363559, 0.68876874, 0.        , 0.0892366 , 1.        ]])

In [ ]:
def reorder(peso,original): #metodo que ordena el arreglo original en base a cada peso
  aux = []
  for i in range(len(peso)):
    index = peso.index(max(peso))
    if(max(peso) != -1):
      aux.append(index)
      peso[index] = -1
  return aux

##Detección de el/los problema/s similar/es

In [ ]:
#claves
problema_similar,arr_peso = mayorSimilitudPeso(cs_clavesnuevas)

if (problema_similar[0] ==  -1):
    print("No hay problemas con claves similares, no se puede recomendar")
else:
  if (len(problema_similar) == 1):
    print("El problema con claves mas similares es Problema ",problema_similar[0]+1)
  else:
    #anio
    problema_anio = cosine_similarity_anio(problema_similar) #posibles problemas a recomendar segun el anio
    for i in range (len(problema_similar)):
      if (i not in problema_anio):
        arr_peso[i] -= 1

    #ratings
    pr = transform_ps(problema_anio, problema_similar) #posibles problemas a recomendar segun los ratings
    problema_ratings = cosine_similarity_ratings(pr)
    for i in range(len(problema_anio)):
      if (i not in problema_ratings):
        arr_peso[i] -= 1

    #eliminar problemas ya puntuados por el profesor
    problemas_rankeados = []
    for i in range(num_columnas):
      if (df_ratings.iloc[id_profesor - 1,i] != 0):
        problemas_rankeados.append(i)

    for i in range(len(problema_similar)):
      if(problema_similar[i] in problemas_rankeados):
        arr_peso[i] = -1

    #final
    indices = reorder(arr_peso.copy(),problema_similar) #problemas a recomendar ordenados segun el peso
    print("Los problemas a recomendar son: ")
    for i in range(len(indices)):
     print(" - Problema ",problema_similar[indices[i]]+1)

Los problemas a recomendar son: 
 - Problema  4
 - Problema  5
 - Problema  1
